In [63]:
import cv2
import numpy as np
import pandas as pd
import os

def process_image(image_path):
    image = cv2.imread(image_path)
    original_image = image.copy()

    # Preprocess the image
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 30, 20)

    # Create a colored version of edges for drawing
    colored_edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

    return image, colored_edges

def adjust_to_rectangle(points):
    centroid = np.mean(points, axis=0)
    angles = np.arctan2(points[:,1] - centroid[1], points[:,0] - centroid[0])
    points = points[np.argsort(angles)]
    
    avg_width = np.mean([np.linalg.norm(points[1]-points[0]), np.linalg.norm(points[2]-points[3])])
    avg_height = np.mean([np.linalg.norm(points[2]-points[1]), np.linalg.norm(points[3]-points[0])])
    
    direction1 = (points[1] - points[0]) / np.linalg.norm(points[1]-points[0])
    direction2 = (points[3] - points[0]) / np.linalg.norm(points[3]-points[0])
    
    points[1] = points[0] + direction1 * avg_width
    points[2] = points[1] + direction2 * avg_height
    points[3] = points[0] + direction2 * avg_height
    
    return points

def save_preprocessed_and_labeled_images(image_path, colored_edges, points):
    # Derive the directory where the original image is
    directory = os.path.dirname(image_path)
    
    # Save preprocessed image
    preprocessed_filename = os.path.join(directory, f"{base_name}_prep.jpg")
    success_prep = cv2.imwrite(preprocessed_filename, colored_edges)
    
    # Draw rectangle on original image and save
    for i in range(4):
        cv2.line(image, tuple(points[i]), tuple(points[(i+1)%4]), (0, 255, 0), 2)
    labeled_filename = os.path.join(directory, f"{base_name}_rect.jpg")
    success_label = cv2.imwrite(labeled_filename, image)

    if success_prep:
        print(f"Successfully saved: {preprocessed_filename}")
    else:
        print(f"Failed to save: {preprocessed_filename}")

    if success_label:
        print(f"Successfully saved: {labeled_filename}")
    else:
        print(f"Failed to save: {labeled_filename}")


def select_points(event, x, y, flags, param):
    global points, colored_edges

    if event == cv2.EVENT_LBUTTONDOWN:
        if len(points) < 4:
            cv2.circle(colored_edges, (x, y), 5, (0, 0, 255), -1)
            points.append([x, y])
            
            if len(points) == 4:
                points = adjust_to_rectangle(np.array(points))
                
                # Connect the points to form a closed area
                for i in range(4):
                    cv2.line(colored_edges, tuple(points[i]), tuple(points[(i+1)%4]), (0, 255, 0), 2)
                
                # Create a DataFrame
                df = pd.DataFrame(points, columns=[f'{base_name}_x', f'{base_name}_y'])
                #print(df)
                
                df_flat = df.values.flatten()
                df_flat = pd.DataFrame(df_flat.reshape(1, -1), columns=['top_left_x', 'top_left_y', 'top_right_x', 'top_right_y', 'bottom_right_x', 'bottom_right_y', 'bottom_left_x', 'bottom_left_y'])
                csv_data = df_flat.to_csv(index=False, sep=',')
                print(csv_data)
                
                save_preprocessed_and_labeled_images(image_path, colored_edges.copy(), points)
                
            cv2.imshow("Preprocessed Image", colored_edges)

#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\322.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\322_sal.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\326.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\326_pfa.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\327.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\327_pfa.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\328.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\328_sal.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\330.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\330_sal.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\332.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\332_sal.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\334.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\334_pfa.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\335.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\335_pfa.png'
#image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\336.png'
image_path = r'H:\DANNCE\6cam_behavior\topviewcam\mp4\336_pfa.png'
image, colored_edges = process_image(image_path)
points = []

# Derive the base filename at the top-level scope
base_filename = os.path.basename(image_path)
base_name, ext = os.path.splitext(base_filename)

cv2.namedWindow("Preprocessed Image")
cv2.setMouseCallback("Preprocessed Image", select_points)
cv2.imshow("Preprocessed Image", colored_edges)
cv2.waitKey(0)
cv2.destroyAllWindows()


top_left_x,top_left_y,top_right_x,top_right_y,bottom_right_x,bottom_right_y,bottom_left_x,bottom_left_y
398,142,932,142,928,403,394,403

Successfully saved: H:\DANNCE\6cam_behavior\topviewcam\mp4\336_pfa_prep.jpg
Successfully saved: H:\DANNCE\6cam_behavior\topviewcam\mp4\336_pfa_rect.jpg
